In [1]:
import dash
from dash import dcc, html
import dash_leaflet as dl
import geopandas as gpd
import json
import pandas as pd

In [ ]:


# if __name__ == "__main__":
#     app.run_server(debug=True)

In [ ]:

job_outlook_data

,NOC_Code,NOC Title,Outlook,Employment Trends,Release Date,Province,Economic Region Code,Economic Region Name,LANG
0,NOC_00010,Legislators,undetermined,<P>An employment outlook has not been assigned...,2024-11-21,NL,1000,Newfoundland and Labrador,EN
1,NOC_00010,Legislators,undetermined,<P>An employment outlook has not been assigned...,2024-11-21,NL,1010,Avalon Peninsula,EN
2,NOC_00010,Legislators,undetermined,<P>An employment outlook has not been assigned...,2024-11-21,NL,1020,South Coast - Burin Peninsula,EN
3,NOC_00010,Legislators,undetermined,<P>An employment outlook has not been assigned...,2024-11-21,NL,1030,West Coast - Northern Peninsula - Labrador,EN
4,NOC_00010,Legislators,undetermined,<P>An employment outlook has not been assigned...,2024-11-21,NL,1040,Notre Dame - Central Bonavista Bay,EN
...,...,...,...,...,...,...,...,...,...
43855,NOC_95109,"Other labourers in processing, manufacturing a...",undetermined,<P>An employment outlook has not been assigned...,2024-12-03,BC,5970,Nechako,EN
43856,NOC_95109,"Other labourers in processing, manufacturing a...",undetermined,<P>An employment outlook has not been assigned...,2024-12-03,BC,5980,Northeast,EN
43857,NOC_95109,"Other labourers in processing, manufacturing a...",undetermined,<P>An employment outlook has not been assigned...,2024-11-21,YK,6010,Yukon,EN
43858,NOC_95109,"Other labourers in processing, manufacturing a...",undetermined,<P>An employment outlook has not been assigned...,2024-12-04,NT,6110,Northwest Territories,EN


In [ ]:
import dash
from dash import dcc, html
import dash_leaflet as dl
import geopandas as gpd
import json
import pandas as pd

# Load shapefile
gdf = gpd.read_file("./data/ler_000b16a_e.shp")

gdf = gdf.to_crs(epsg=4326)

# Convert to GeoJSON
geojson = json.loads(gdf.to_json())

job_outlook_data = pd.read_excel('./data/20242026_outlook_n21_en_250117.xlsx')

app = dash.Dash(__name__)

# Define colors for each outlook
outlook_colors = {
    'very good': 'green',
    'good': 'blue',
    'fair': 'yellow',
    'limited': 'orange',
    'undetermined': 'red'
}

# Filter job_outlook_data based on Outlook values
filtered_data = job_outlook_data[job_outlook_data['Outlook'].isin(outlook_colors.keys())]

# Extract coordinates for each Economic Region Name
region_coords = gdf[['ERNAME', 'geometry']].set_index('ERNAME').to_dict()['geometry']

# Create markers for each region
markers = []
for _, row in filtered_data.iterrows():
    region_name = row['Economic Region Name']
    outlook = row['Outlook']
    if region_name in region_coords:
        coords = region_coords[region_name].centroid.coords[0]
        markers.append(dl.Marker(position=[coords[1], coords[0]], children=dl.Tooltip(region_name), icon={
            "iconUrl": f"http://maps.google.com/mapfiles/ms/icons/{outlook_colors[outlook]}-dot.png"
        }))

# Add shaded background for provinces/territories
shaded_regions = []
for region_name, geometry in region_coords.items():
    shaded_regions.append(dl.GeoJSON(data=json.loads(gpd.GeoSeries([geometry]).to_json()), style={
        "fillColor": "grey",
        "fillOpacity": 0.2,
        "color": "grey",
        "weight": 1
    }))

# Create a legend manually
legend = html.Div([
    html.Div([html.Span(style={'backgroundColor': color, 'display': 'inline-block', 'width': '20px', 'height': '20px'}), html.Span(f' {outlook}')]) 
    for outlook, color in outlook_colors.items()
], style={'position': 'absolute', 'bottom': '10px', 'left': '10px', 'backgroundColor': 'white', 'padding': '10px', 'border': '1px solid black'})

# Update the app layout to include the new markers, shaded regions, and legend
app.layout = html.Div([
    dl.Map([
        dl.TileLayer(),
        dl.LayerGroup(markers),
        dl.LayerGroup(shaded_regions),
        dl.GeoJSON(data=geojson, style={"color": "grey", "weight": 2}),
    ], center=[gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()], zoom=5, style={"height": "600px"}),
    legend
])

if __name__ == "__main__":
    app.run_server(debug=True)

C:\Users\lucas\AppData\Local\Temp\ipykernel_10364\2777511592.py:52: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


